# Inspecting the eigensolver steps

In [1]:
%load_ext autoreload
%autoreload 2

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import os, json
from pg_utils.pg_model import base, base_utils, core, expansion, forcing
from pg_utils.sympy_supp import simplify as simp_supp
from pg_utils import eigen
from IPython.display import display, Math, Markdown

def display_equality(lhs, rhs, _filter=lambda lhs, rhs, idx: True):
    display(*(sp.Eq(lhs_tmp, rhs[idx]) for idx, lhs_tmp in enumerate(lhs) if _filter(lhs_tmp, rhs[idx], idx)))

def display_eq_collection(eqs, _filter=lambda eq, idx: True):
    display(*(eq for idx, eq in enumerate(eqs) if _filter(eq, idx)))

---
## Background field

In [ ]:
from pg_utils.pg_model import bg_fields

bg_cfg = bg_fields.Background_S_l2_n2()

display_equality(core.U0_vec, bg_cfg.U0_val)
display_equality(core.B0_vec, bg_cfg.B0_val)

In [ ]:
bg_pg = base_utils.assemble_background(bg_cfg.B0_val)
# bg_pg.apply(lambda expr: expr.subs({core.H_s: core.H, core.H_s**2: core.H**2}).simplify(), inplace=True)
bg_pg.apply(lambda expr: expr.subs({core.H: core.H_s}).simplify(), inplace=True)
display_equality(core.pgvar_bg, bg_pg)

In [ ]:
bg_placeholders = base.map_collection(core.pgvar_bg, base_utils.extract_symbols(core.pgvar_bg))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in bg_pg._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(sp.Eq(core.pgvar_bg[fname].subs(bg_placeholders), bg_pg[fname].subs(placeholder_h)))
    print(o_str)

---
## Equations

In [ ]:
placeholders_map = {
    **base.map_collection(core.pgvar_ptb, base_utils.extract_symbols(core.pgvar_ptb)),
    **base.map_collection(core.pgvar_bg, base_utils.extract_symbols(core.pgvar_bg))
}
placeholder_h = {core.H: sp.Symbol("H")}
for fname in eqs_pg_lin._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(eqs_pg_lin[fname].subs(placeholder_h))
    for var_old, var_new in placeholders_map.items():
        o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
    # print(o_str)
    display(Math(o_str))

### PG system

In [10]:
# eqs_file = "./out/eigen/Toroidal_Quadrupole/eqs_pg.json"
# eqs_file = "./out/eigen/Poloidal_Dipole/eqs_pg.json"
eqs_file = './out/eigen/S_L2_N2/eqs_pg_ideal.json'
with open(eqs_file, 'r') as fread:
    eqs_pg = base.CollectionPG.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)

Convert to ODE in cylindrical radius $s$ (equiv. convert to frequency-wavenumber domain)

In [11]:
ode_s_fd = eigen.to_fd_ode_pg(eqs_pg, core.pgvar_ptb)

Some further manipulations before output

In [ ]:
from pg_utils.sympy_supp import simplify
from pg_utils.numerics import utils as nutils

# bg_subs = {bg_cfg.params[0]: sp.sqrt(sp.pi/3)*bg_cfg.params[0]}
bg_subs = {bg_cfg.params[0]: 2*sp.sqrt(sp.pi/5)/3}

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    # num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    # num = num.subs([(sp.diff(core.H, (core.s, 2)), -/core.H**3), (sp.diff(core.H, core.s), -core.s/core.H)])
    # num = num.subs({core.H: core.H_s}).doit().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    num = num.subs({core.H: core.H_s}).simplify().factor()
    out = num/denom
    num, denom = sp.fraction(out)
    num = num.simplify().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    denom = denom.simplify().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

# eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
# simplify.process_leaf_node(eq_grouped, common_denom_simplify)
# eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)

proc_timer = nutils.ProcTimer(start=True)

# factor = 1/sp.I/expansion.omega
factor = 1

def factoring(fname, eq):
    
    if fname == "Psi":
        # pass
        expr_grouped = (eq.rhs.subs(bg_subs)*factor).expand().collect([var for var in expansion.pgvar_s if var is not None], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        proc_timer.flag(loginfo=fname, print_str=True, increment=True)
        return sp.Eq((eq.lhs*factor).expand(), sp.Add(*[term*coeff for term, coeff in expr_grouped.items()]))
    
    elif fname == "Br_b":
        proc_timer.flag(loginfo=fname, print_str=True, increment=True)
        return None
    
    else:
        expr_grouped = (eq.rhs.subs(bg_subs)*factor).expand().collect([expansion.pgvar_s.Psi,], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        proc_timer.flag(loginfo=fname, print_str=True, increment=True)
        return sp.Eq(
            (eq.lhs*factor).expand(), 
            # (eq.rhs*factor).expand().collect(expansion.pgvar_s.Psi)
            factor*sp.Add(*[term*coeff for term, coeff in expr_grouped.items()])
        )

ode_s_fd.apply(factoring, inplace=True, metadata=True)
proc_timer.print_elapse(increment=False)
display_eq_collection(ode_s_fd)

In [ ]:
placeholders_map = base.map_collection(expansion.pgvar_s, base_utils.extract_symbols(core.pgvar_ptb))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in ode_s_fd._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(ode_s_fd[fname].subs(placeholder_h))
    for var_old, var_new in placeholders_map.items():
        o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
    # print(o_str)
    display(Math(o_str))

Convert directly to reduced form in Fourier domain

In [ ]:
# reduce_map = {expansion.pgvar_s[fname]: ode_s_fd[fname].rhs if fname not in ("Bp_p", "Bp_m") else sp.S.Zero for fname in ode_s_fd._field_names if fname not in ("Psi", "Br_b")}
reduce_map = {expansion.pgvar_s[fname]: ode_s_fd[fname].rhs/sp.I/expansion.omega for fname in ode_s_fd._field_names if fname not in ("Psi", "Br_b")}
# ode_psi = sp.Eq((ode_s_fd.Psi.lhs - ode_s_fd.Psi.rhs.subs(reduce_map)).subs({core.H: core.H_s}).doit().subs({core.H_s: core.H, core.H_s**2: core.H**2}).expand(), sp.S.Zero)
ode_psi = sp.Eq((ode_s_fd.Psi.lhs - ode_s_fd.Psi.rhs.subs(reduce_map)).doit().subs({core.H_s: core.H, core.H_s**2: core.H**2}).expand(), sp.S.Zero)
(ode_psi.lhs*sp.I*expansion.omega).expand()

In [ ]:
def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree((ode_psi.lhs*expansion.omega).expand(), (expansion.pgvar_s.Psi, expansion.omega, expansion.m))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), ode_psi.rhs)
eq_grouped

In [ ]:
denom_poles = sp.fraction(leading_cf)[0]/(sp.I*bg_cfg.params[0]**2)

In [ ]:
denom_func = sp.lambdify([core.s,], denom_poles, modules=['numpy', 'scipy'])

In [ ]:
s = np.linspace(0, 1, num=100)
fig, ax = plt.subplots(figsize=(9, 6))
ax.plot(s, denom_func(s))
ax.grid(which='both')
plt.show()

In [ ]:
ord_max = 4
leading_cf = ode_psi.lhs.coeff(sp.diff(expansion.pgvar_s.Psi, (core.s, ord_max))).together().subs({core.H_s: core.H, core.H_s**2: core.H**2})
ode_psi = sp.Eq((ode_psi.lhs/leading_cf).expand(), ode_psi.rhs/leading_cf)
sp.Eq(ode_psi.lhs.collect(expansion.pgvar_s.Psi), ode_psi.rhs)

from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree(ode_psi.lhs, (expansion.pgvar_s.Psi, expansion.omega))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), ode_psi.rhs)
eq_grouped

In [ ]:
o_str = sp.latex(eq_grouped.subs(placeholder_h))
for var_old, var_new in placeholders_map.items():
    o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
print(o_str)

### Transformed variables

In [ ]:
eqs_file = "./out/eigen/Poloidal_Dipole/eqs_cg.json"
with open(eqs_file, 'r') as fread:
    eqs_cg = base.CollectionConjugate.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)
display_eq_collection(eqs_cg)
# eqs_cg.Psi

In [ ]:
ode_s_fd_cg = eigen.to_fd_ode_cg(eqs_cg, core.cgvar_ptb)

Some further manipulations before output

In [ ]:
from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

# eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
# simplify.process_leaf_node(eq_grouped, common_denom_simplify)
# eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)

# factor = 1/sp.I/expansion.omega
factor = 1
def factoring(fname, eq):
    if fname != "Psi":
        expr_grouped = (eq.rhs*factor).expand().collect([expansion.cgvar_s.Psi,], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq(
            (eq.lhs*factor).expand(), 
            # (eq.rhs*factor).expand().collect(expansion.pgvar_s.Psi)
            factor*sp.Add(*[term*coeff for term, coeff in expr_grouped.items()])
        )
    else:
        expr_grouped = (eq.rhs*factor).expand().collect([var for var in expansion.cgvar_s if var is not None], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq((eq.lhs*factor).expand(), sp.Add(*[term*coeff for term, coeff in expr_grouped.items()]))

ode_s_fd_cg.apply(factoring, inplace=True, metadata=True)
display_eq_collection(ode_s_fd_cg)

In [ ]:
forcing = ode_s_fd_cg.M_zp.rhs.subs(
    {expansion.cgvar_s.Psi: core.s**expansion.m*core.H**3*sp.jacobi(expansion.n, sp.Rational(3, 2), expansion.m, 2*core.s**2 - 1)})

In [ ]:
forcing = forcing.doit().subs({sp.diff(core.H, core.s): -core.s/core.H}).doit().subs({sp.diff(core.H, core.s): -core.s/core.H})

In [ ]:
forcing.expand()

In [ ]:
forcing.expand().collect(core.H)

In [ ]:
placeholders_map = base.map_collection(expansion.cgvar_s, base_utils.extract_symbols(core.cgvar_ptb))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in ode_s_fd_cg._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(ode_s_fd_cg[fname].subs(placeholder_h))
    for var_old, var_new in placeholders_map.items():
        o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
    # print(o_str)
    display(Math(o_str))

In [ ]:
map_cg_pg = core.map_conjugate_to_pg(core.cgvar_ptb, core.pgvar_ptb)

In [ ]:
eqs_pg_from_cg = eqs_cg.apply(lambda eq: eq.subs(map_cg_pg))
eqs_pg_from_cg = core.conjugate_to_PG(eqs_pg_from_cg).apply(lambda eq: eq.doit().expand())

In [ ]:
display_eq_collection(eqs_pg_from_cg)

In [ ]:
ode_s_fd_from_cg = eigen.to_fd_ode_pg(eqs_pg_from_cg, core.pgvar_ptb)

Some further manipulations before output

In [ ]:
from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

# eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
# simplify.process_leaf_node(eq_grouped, common_denom_simplify)
# eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)

# factor = 1/sp.I/expansion.omega
factor = 1
def factoring(fname, eq):
    if fname != "Psi":
        expr_grouped = (eq.rhs*factor).expand().collect([expansion.pgvar_s.Psi,], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq(
            (eq.lhs*factor).expand(), 
            # (eq.rhs*factor).expand().collect(expansion.pgvar_s.Psi)
            factor*sp.Add(*[term*coeff for term, coeff in expr_grouped.items()])
        )
    else:
        expr_grouped = (eq.rhs*factor).expand().collect([var for var in expansion.pgvar_s if var is not None], evaluate=False)
        for term in expr_grouped:
            expr_grouped[term] = common_denom_simplify(expr_grouped[term])
        return sp.Eq((eq.lhs*factor).expand(), sp.Add(*[term*coeff for term, coeff in expr_grouped.items()]))

ode_s_fd_from_cg.apply(factoring, inplace=True, metadata=True)
# display_eq_collection(ode_s_fd)

In [ ]:
placeholders_map = base.map_collection(expansion.pgvar_s, base_utils.extract_symbols(core.pgvar_ptb))
placeholder_h = {core.H: sp.Symbol("H")}
for fname in ode_s_fd._field_names:
    if fname == "Br_b":
        continue
    o_str = sp.latex(ode_s_fd_from_cg[fname].subs(placeholder_h))
    for var_old, var_new in placeholders_map.items():
        o_str = o_str.replace(sp.latex(var_old), sp.latex(var_new))
    # print(o_str)
    display(Math(o_str))

Is the equation equivalent to the one by PG?

In [ ]:
for fname in ode_s_fd._field_names:
    if ode_s_fd_from_cg[fname].rhs.subs({core.H: core.H_s}).equals(ode_s_fd[fname].rhs.subs({core.H: core.H_s})) \
        and ode_s_fd_from_cg[fname].lhs.equals(ode_s_fd[fname].lhs):
        display(Markdown("${}$: Equivalent".format(core.pgvar_ptb[fname])))
    else:
        display(Markdown("${}$: NOT equivalent!, difference = ${}$".format(
            core.pgvar_ptb[fname], 
            sp.latex(ode_s_fd_from_cg[fname].rhs.subs({core.H: core.H_s}) - ode_s_fd[fname].rhs.subs({core.H: core.H_s}))
        )))

### Reduced dimensional system

In [ ]:
str(sp.Rational(1, 2))

In [ ]:
eqs_file = "./out/eigen/Toroidal_Quadrupole/eqs_reduced_v0.json"
with open(eqs_file, 'r') as fread:
    eqs_reduced = base.LabeledCollection.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)

display_eq_collection(eqs_reduced)

In [ ]:
eqs_file = "./out/eigen/Toroidal_Quadrupole/eqs_reduced.json"
with open(eqs_file, 'r') as fread:
    eqs_reduced = base.LabeledCollection.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)

display_eq_collection(eqs_reduced)

In [ ]:
eqs_2ord = sp.Eq(sp.diff(eqs_reduced.Psi.lhs, core.t), sp.diff(eqs_reduced.Psi.rhs, core.t).subs({eqs_reduced.F_ext.lhs: eqs_reduced.F_ext.rhs}))
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

In [ ]:
def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree(eq_rad.lhs, (expansion.pgvar_s.Psi, expansion.omega, expansion.m))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad.rhs)
eq_grouped

In [ ]:
eqs_2ord = sp.Eq(sp.diff(eqs_reduced.Psi.lhs, core.t), sp.diff(eqs_reduced.Psi.rhs, core.t).subs({eqs_reduced.F_ext.lhs: eqs_reduced.F_ext.rhs}))
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

In [ ]:
def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2}).collect(expansion.m)
    return num/denom.factor()

eq_grouped = simplify.recursive_collect_tree(eq_rad.lhs, (expansion.pgvar_s.Psi, expansion.omega, expansion.m))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad.rhs)
eq_grouped

In [ ]:
eqs_2ord = sp.Eq(
    sp.diff(eqs_reduced.Psi.lhs, core.t),
    sp.diff(eqs_reduced.Psi.rhs, core.t).subs({eqs_reduced.F_ext.lhs: eqs_reduced.F_ext.rhs}).doit().expand()
)
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

### Second-order form

In [ ]:
eqs_file = "./out/eigen/Poloidal_Dipole/eqs_pg.json"
with open(eqs_file, 'r') as fread:
    eqs_pg = base.CollectionPG.deserialize(json.load(fread)["equations"], parser=sp.parse_expr)
eqs_pg_select = eqs_pg.copy()
# for i_eq in range(14, 21):
#     eqs_pg_select[i_eq] = sp.Eq(eqs_pg_select[i_eq].lhs, sp.S.Zero)
eqs_pg_select[16] = sp.Eq(eqs_pg_select[16].lhs, sp.S.Zero)
eqs_pg_select[19] = sp.Eq(eqs_pg_select[19].lhs, sp.S.Zero)

eqs_2ord = eigen.reduce_eqsys_to_psi(eqs_pg_select, verbose=5)
eq_rad = eigen.to_fd_ode_psi(eqs_2ord)
eq_rad = sp.Eq(eq_rad.lhs - eq_rad.rhs, sp.S.Zero)

In [ ]:
eqs_pg_select[0]

Further simplifications: standard ODE form

In [ ]:
eq_rad

In [ ]:
ord_max = 4
leading_cf = eq_rad.lhs.coeff(sp.diff(expansion.pgvar_s.Psi, (core.s, ord_max))).together().subs({core.H_s: core.H, core.H_s**2: core.H**2})
eq_rad_std = sp.Eq((eq_rad.lhs/leading_cf).expand(), eq_rad.rhs/leading_cf)
sp.Eq(eq_rad_std.lhs.collect(expansion.pgvar_s.Psi), eq_rad_std.rhs)

Group the coefficients

In [ ]:
from pg_utils.sympy_supp import simplify

def common_denom_simplify(expr):
    expr = expr.together()
    num, denom = sp.fraction(expr)
    num = num.subs({core.H: core.H_s}).simplify().factor().subs({core.H_s: core.H, core.H_s**2: core.H**2})
    return num/denom

eq_grouped = simplify.recursive_collect_tree(eq_rad_std.lhs, (expansion.pgvar_s.Psi, expansion.omega))
simplify.process_leaf_node(eq_grouped, common_denom_simplify)
eq_grouped = sp.Eq(simplify.recursive_eval_tree(eq_grouped), eq_rad_std.rhs)
eq_grouped

Output

In [ ]:
sp.print_latex(eq_grouped)

## Matrix elements

In [ ]:
from pg_utils.numerics import matrices as nmatrix

matrix_file = "./out/eigen/Malkus/Transformed/matrix_expr.json"
with open(matrix_file, 'r') as fread:
    matrix_obj = json.load(fread)

M_expr = expansion.SystemMatrix.deserialize(matrix_obj["M"])
K_expr = expansion.SystemMatrix.deserialize(matrix_obj["K"])
par_list = [sp.parse_expr(par) for par in matrix_obj["params"]]

In [ ]:
iprod = nmatrix.InnerQuad_GaussJacobi(M_expr["B_em", "B_em"], automatic=True)

In [ ]:
iprod.deduce_params_outer(51, 51)

In [ ]:
M_expr["B_em", "B_em"]._opd_A

In [ ]:
nmatrix.InnerQuad_GaussJacobi.get_powers(expansion.xi, M_expr["B_em", "B_em"]._opd_A.doit())

In [ ]:
opd_A_terms = nmatrix.InnerQuad_GaussJacobi.get_powers(expansion.xi, K_expr["F_ext", "Psi"]._opd_A, return_expr=True)
if not isinstance(opd_A_terms, list):
    opd_A_terms = [opd_A_terms]

for term in opd_A_terms:
    display(term[1])
    print(term[0])

In [ ]:
opd_B_terms = nmatrix.InnerQuad_GaussJacobi.get_powers(expansion.xi, K_expr["F_ext", "Psi"]._opd_B.doit(), return_expr=True)
if not isinstance(opd_B_terms, list):
    opd_B_terms = [opd_B_terms]

In [ ]:
for term in opd_B_terms:
    if term[0][0] > -1:
        continue
    display(term[1])
    print(term[0])

In [ ]:
term_singular = sp.S.Zero
for term in opd_B_terms:
    if term[0][0] <= -1:
        term_singular += term[1]

In [ ]:
term_singular

In [ ]:
term_singular.subs({sp.Symbol(r"p_2", positive=True): 2 - sp.Symbol(r"p_1", positive=True)}).simplify().subs({2 - sp.Symbol(r"p_1", positive=True): sp.Symbol(r"p_2", positive=True)})

In [ ]:
[term[0] for term in opd_B_terms]

In [ ]:
with open("./out/symbolic/eqs_pg_lin.json", 'r') as fread:
    eqs = base.LabeledCollection.load_json(fread, parser=sp.parse_expr)

In [ ]:
eqs.Psi

In [ ]:
eqs.dBp_dz_e

In [ ]:
(sp.diff(core.s*sp.diff(core.s**expansion.m*core.H_s**3, core.s), core.s)/expansion.H_s).expand()

In [ ]:
sp.diff(core.s**expansion.m, core.s).subs({expansion.m: 0})

In [ ]:
(sp.diff(core.s**expansion.m*core.H_s**3, core.s)/expansion.H_s).expand()

---
## General equations inspection

In [12]:
with open('./out/symbolic/eqs-pg__boundIE-Bcyl.json', 'r') as fread:
    eqs_pg = base.LabeledCollection.load_json(fread, parser=sp.parse_expr)

In [11]:
with open('./out/symbolic/eqs-pg__boundIE-Bcyl.json', 'r') as fread:
    eqs_pg_lin = base.LabeledCollection.load_json(fread, parser=sp.parse_expr)
    
zero_bg_u_map = {
    core.U0_vec[0]: 0,
    core.U0_vec[1]: 0,
    core.U0_vec[2]: 0
}
eqs_pg_lin.apply(lambda eq: sp.Eq(eq.lhs, eq.rhs.subs(zero_bg_u_map).doit()) if eq is not None else None, inplace=True)

In [ ]:
display_eq_collection(eqs_pg)

---
## Equation sequence inspection

In [160]:
# Original linearised equation
with open('./out/symbolic/eqs-cg__boundIE-Bcyl__lin.json', 'r') as fread:
    eqs_pg_lin = base.LabeledCollection.load_json(fread, parser=sp.parse_expr)
eqs_pg_lin, _ = eigen.assemble_forcing(eqs_pg_lin, 'Lorentz', timescale='Alfven', verbose=0)
eqs_pg_lin.Psi = sp.Eq(eqs_pg_lin.Psi.lhs, eqs_pg_lin.Psi.rhs.subs(forcing.force_explicit_lin_cg).doit().expand())
    
# Background-specific eigenvalue equation
dir_bg = './out/eigen/S1_T2_std/'
with open(os.path.join(dir_bg, 'eqs_ideal_canonical_vs.json'), 'r') as fread:
    load_array = json.load(fread)
    eqs_eigen_bg = base.LabeledCollection.deserialize(load_array['equations'], parser=sp.parse_expr)
    
# Matrix elements
dir_spec = os.path.join(dir_bg, 'Canonical')
with open(os.path.join(dir_spec, 'matrix_expr_ideal_vsHjx.json'), 'r') as fread:
    matrix_obj = json.load(fread)
M_expr = expansion.SystemMatrix.deserialize(matrix_obj["M"])
K_expr = expansion.SystemMatrix.deserialize(matrix_obj["K"])

In [8]:
from pg_utils.pg_model import expand_conjugate_ext as xpd_cfg

eqs_eigen_selected = eqs_eigen_bg.generate_collection(xpd_cfg.field_indexer)
M_simp, K_simp = eigen.equations_to_matrices(eqs_eigen_selected, xpd_cfg.recipe)

In [11]:
%lprun -f eigen.equations_to_matrices eigen.equations_to_matrices(eqs_eigen_selected, xpd_cfg.recipe)

Timer unit: 1e-07 s

Total time: 253.716 s
File: c:\Users\gentle\Documents\Projects\PG-Assim\pg_utils\eigen.py
Function: equations_to_matrices at line 333

Line #      Hits         Time  Per Hit   % Time  Line Contents
   333                                           def equations_to_matrices(eqs: base.LabeledCollection, 
   334                                               xpd_recipe: xpd.ExpansionRecipe, inplace: bool = False, 
   335                                               verbose: int = 0) -> List[xpd.SystemMatrix]:
   336                                               """Collect matrix elements
   337                                               
   338                                               :param base.LabeledCollection eqs: collection of all equations
   339                                               :param expansion.ExpansionRecipe xpd_recipe: expansion recipe
   340                                               :param bool inplace: whether to modify the eqs inp

### Show element

In [161]:
eqs_pg_lin._field_names

['Psi',
 'M_1',
 'M_p',
 'M_m',
 'M_zp',
 'M_zm',
 'zM_1',
 'zM_p',
 'zM_m',
 'B_ep',
 'B_em',
 'Bz_e',
 'dB_dz_ep',
 'dB_dz_em',
 'Br_b',
 'B_pp',
 'B_pm',
 'Bz_p',
 'B_mp',
 'B_mm',
 'Bz_m']

In [173]:
name_eqn = 'M_p'
name_term = 'Psi'

k = sp.Symbol('k', positive=True)
H1 = sp.Symbol('H', positive=True)
Jac = 1/(4*sp.sqrt((1 + expansion.xi)/2))
sub_ab = {(1 - expansion.xi)/2: H1**2, (1 + expansion.xi)/2: core.s**2, (expansion.xi + 1)/2: core.s**2, 1 - expansion.xi: 2*H1**2, expansion.xi - 1: -2*H1**2}

In [174]:
term_pg_lin = simp_supp.summands_dep(eqs_pg_lin[name_eqn].rhs, core.cgvar_ptb[name_term])
term_eigen = simp_supp.summands_dep(eqs_eigen_bg[name_eqn].rhs, core.cgvar_ptb[name_term])

element_eigen = K_expr[name_eqn, name_term]
if isinstance(element_eigen, expansion.InnerProduct1D):
    arg_simp = element_eigen.opd_B
else:
    arg_simp = element_eigen

# element_eigen = K_expr[name_eqn, name_term]
# if isinstance(element_eigen, expansion.InnerProduct1D):
#     arg_simp = element_eigen._opd_B.subs({expansion.m: sp.Integer(2) + k}).doit().subs({k: expansion.m - sp.Integer(2)})
#     arg_simp_terms = simp_supp.collect_jacobi(arg_simp, evaluate=False)
#     arg_simp = sp.Add(*[cf.simplify()*basis for basis, cf in arg_simp_terms.items()])
#     # (element_eigen.args[1].subs({expansion.m: sp.Integer(2) + k}).doit().subs({k: expansion.m - sp.Integer(2)}).xreplace(sub_ab)/Jac.xreplace(sub_ab)).simplify().xreplace(sub_ab).simplify()
# else:
#     arg_simp = element_eigen

display(term_pg_lin, term_eigen, arg_simp)

-I*\overline{M_+}^0(s, \phi, t)*Derivative(\psi(s, \phi, t), (s, 2))/H(s) - I*\overline{M_1}^0(s, \phi, t)*Derivative(\psi(s, \phi, t), (s, 2))/H(s) + I*\overline{M_+}^0(s, \phi, t)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), s)/H(s)**2 + I*\overline{M_1}^0(s, \phi, t)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), s)/H(s)**2 + I*\overline{M_+}^0(s, \phi, t)*Derivative(\psi(s, \phi, t), s)/(s*H(s)) + I*\overline{M_1}^0(s, \phi, t)*Derivative(\psi(s, \phi, t), s)/(s*H(s)) + 2*\overline{M_1}^0(s, \phi, t)*Derivative(\psi(s, \phi, t), \phi, s)/(s*H(s)) + Derivative(\overline{M_+}^0(s, \phi, t), \phi)*Derivative(\psi(s, \phi, t), s)/(s*H(s)) - Derivative(\overline{M_+}^0(s, \phi, t), s)*Derivative(\psi(s, \phi, t), \phi)/(s*H(s)) - \overline{M_1}^0(s, \phi, t)*Derivative(H(s), s)*Derivative(\psi(s, \phi, t), \phi)/(s*H(s)**2) - I*\overline{M_+}^0(s, \phi, t)*Derivative(\psi(s, \phi, t), (\phi, 2))/(s**2*H(s)) - 2*\overline{M_1}^0(s, \phi, t)*Derivative(\psi(s, \phi, t), \phi)/(s**2*

-128*s**6*Derivative(\psi(s, \phi, t), \phi)/3 + 128*s**5*H(s)**2*Derivative(\psi(s, \phi, t), \phi, s)/3 - 16*s**5*Derivative(\psi(s, \phi, t), s)/5 - 256*s**4*H(s)**2*Derivative(\psi(s, \phi, t), \phi)/15 + 128*I*s**4*H(s)**2*Derivative(\psi(s, \phi, t), (\phi, 2))/3 - 16*s**4*H(s)**2*Derivative(\psi(s, \phi, t), (s, 2))/5 + 256*s**4*Derivative(\psi(s, \phi, t), \phi)/3 - 48*I*s**4*Derivative(\psi(s, \phi, t), \phi)/5 + 256*s**3*H(s)**4*Derivative(\psi(s, \phi, t), \phi, s)/5 + 32*s**3*H(s)**2*Derivative(\psi(s, \phi, t), s)/25 - 256*s**3*H(s)**2*Derivative(\psi(s, \phi, t), \phi, s)/3 + 16*s**3*Derivative(\psi(s, \phi, t), s)/5 - 6*I*s**3*Derivative(\psi(s, \phi, t), s)/25 - 128*s**2*H(s)**4*Derivative(\psi(s, \phi, t), \phi)/35 + 256*I*s**2*H(s)**4*Derivative(\psi(s, \phi, t), (\phi, 2))/5 - 48*s**2*H(s)**4*Derivative(\psi(s, \phi, t), (s, 2))/25 + 256*s**2*H(s)**2*Derivative(\psi(s, \phi, t), \phi)/15 + 16*I*s**2*H(s)**2*Derivative(\psi(s, \phi, t), \phi)/5 - 16*s**2*H(s)**2*Deriv

s**3*s**Abs(m)*(64*\ell**2 - 12*I*\ell**2 + 128*\ell*Abs(m) - 24*I*\ell*Abs(m) + 384*\ell - 72*I*\ell + 64*m**2 - 12*I*m**2 + s**2*(-64*\ell**2 - 128*\ell*Abs(m) - 384*\ell - 64*m**2 - 384*Abs(m) - 560) + 384*Abs(m) - 72*I*Abs(m) + 560 - 105*I)*H(s)**5*jacobi(\ell - 2, 7/2, Abs(m) + 2, \xi)/50 + s*s**Abs(m)*(5246*I*\ell*m + 1344*\ell*Abs(m) - 252*I*\ell*Abs(m) + 1344*m**2 - 252*I*m**2 + 5246*I*m*Abs(m) + 13115*I*m + s**6*(-5120*I*\ell*m - 5120*I*m*Abs(m) - 12800*I*m) + s**4*(15360*I*\ell*m + 1344*\ell*Abs(m) + 3360*\ell + 1344*m**2 + 15360*I*m*Abs(m) + 38400*I*m + 6720*Abs(m) + 8400) + s**2*(-15486*I*\ell*m - 2688*\ell*Abs(m) + 252*I*\ell*Abs(m) - 3360*\ell + 630*I*\ell - 2688*m**2 + 252*I*m**2 - 15486*I*m*Abs(m) - 38715*I*m - 10080*Abs(m) + 1260*I*Abs(m) - 8400 + 1575*I) + 3360*Abs(m) - 630*I*Abs(m))*H(s)**3*jacobi(\ell - 1, 5/2, Abs(m) + 1, \xi)/1050 + s**Abs(m)*(-2623*I*m**2 + 2623*I*m*Abs(m) + 672*m - 126*I*m + s**6*(2560*I*m**2 - 2560*I*m*Abs(m) - 15360*I*m) + s**4*(-7680*I*m**2 +

In [ ]:
proc_result = eigen.process_rational_jacobi(
    K_expr['B_pp', 'Psi'],
    xpd_cfg.recipe.base_expr,
    xpd_cfg.recipe.test_expr
)
proc_result.opd_B

1


3*sqrt(2)*I*s**2*s**Abs(m)*(-4*\ell**2 - 8*\ell*Abs(m) - 24*\ell - 4*m**2 - 24*Abs(m) - 35)*H(s)**3*jacobi(\ell - 2, 7/2, Abs(m) + 2, \xi)/40 + 3*sqrt(2)*I*s**Abs(m)*(2*\ell*m - 4*\ell*Abs(m) - 4*m**2 + 2*m*Abs(m) + 5*m + s**2*(-2*\ell*m + 4*\ell*Abs(m) + 10*\ell + 4*m**2 - 2*m*Abs(m) - 5*m + 20*Abs(m) + 25) - 10*Abs(m))*H(s)*jacobi(\ell - 1, 5/2, Abs(m) + 1, \xi)/40 + 3*sqrt(2)*I*s**Abs(m)*(-m**2 + m*Abs(m) - 2*m + s**2*(m**2 - m*Abs(m) + m + 3*Abs(m)) + 2*Abs(m))*H(s)*jacobi(\ell, 3/2, Abs(m), \xi)/(40*s**2)

In [ ]:
sample_expr = K_expr[name_eqn, name_term].opd_B

# powers = symparser.powers_of(sample_expr, core.s, core.H, expansion.xi)
# print(powers)
display(sample_expr)


sqrt(2)*I*s*s**Abs(m - 1)*(2*\ell + 2*Abs(m - 1) + 1)*jacobi(\ell - 1, 1, Abs(m - 1) + 1/2, \xi)/(40*H(s)) + sqrt(2)*I*s**Abs(m - 1)*(-2*m + s**2*(m - 2*Abs(m - 1) - 2) + 2*Abs(m - 1) + 2)*jacobi(\ell, 0, Abs(m - 1) - 1/2, \xi)/(80*s*H(s)**3)

In [ ]:
sample_expr = K_expr[name_eqn, name_term]

# powers = symparser.powers_of(sample_expr, core.s, core.H, expansion.xi)
# print(powers)
display(sample_expr)


InnerProduct1D(s**Abs(m + 2)*H(s)**2*jacobi(\ell', 2, Abs(m + 2) - 1/2, \xi), 45*I*\gamma**2*s**4*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(4*\ell**2 + 8*\ell*Abs(m) + 24*\ell + 4*m**2 + 24*Abs(m) + 35)*H(s)**2*jacobi(\ell - 2, 7/2, Abs(m) + 2, \xi)/(4*pi) - 45*I*\gamma**2*s**2*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(-2*\ell*m*s**2 + 2*\ell*m + 4*\ell*s**2*Abs(m) + 10*\ell*s**2 - 4*\ell*Abs(m) + 4*m**2*s**2 - 4*m**2 - 2*m*s**2*Abs(m) - 5*m*s**2 + 2*m*Abs(m) + 5*m + 20*s**2*Abs(m) + 25*s**2 - 10*Abs(m))*jacobi(\ell - 1, 5/2, Abs(m) + 1, \xi)/(4*pi) - 45*I*\gamma**2*s**Abs(m)*(s - 1)*(s + 1)*(25*m**2*s**6 - 65*m**2*s**4 + 59*m**2*s**2 - 19*m**2 - 25*m*s**6*Abs(m) + 50*m*s**6 + 65*m*s**4*Abs(m) - 140*m*s**4 - 59*m*s**2*Abs(m) + 120*m*s**2 + 19*m*Abs(m) - 38*m + 75*s**6*Abs(m) - 70*s**4*Abs(m) - 23*s**2*Abs(m) + 38*Abs(m))*jacobi(\ell, 3/2, Abs(m), \xi)/(4*pi), 1, s, 0, 1)

In [ ]:
print(len(sp.latex(sample_expr)))

1479


In [37]:
%%time

arg_expr = simp_supp.collect_jacobi(K_expr[name_eqn, name_term]._opd_B, evaluate=False)
arg_expr_proc = dict()
for basis, coeff in arg_expr.items():
    factorised = coeff.together().factor()
    mul_factors = list()
    add_factors = sp.S.One
    for factor in factorised.args:
        if isinstance(factor, sp.Add) and (core.s in factor.atoms(sp.Symbol) or core.H in factor.atoms(sp.Symbol)):
            add_factors *= factor
        else:
            mul_factors.append(factor)
    add_factors = add_factors.subs({core.H: core.H_s}).expand()
    add_factors = simp_supp.horner_delayed(add_factors, core.s)
    mul_factors.append(add_factors)
    mul_factors = sp.Mul(*mul_factors)
    # display(mul_factors.subs({expansion.m: 2}, evaluate=False))
    arg_expr_proc[basis] = factorised
opd_B_rewrite = sp.Add(*[cf.factor()*basis for basis, cf in arg_expr_proc.items()], evaluate=False)
opd_B_rewrite

CPU times: total: 297 ms
Wall time: 502 ms


45*I*\gamma**2*s**4*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(4*\ell**2 + 8*\ell*Abs(m) + 24*\ell + 4*m**2 + 24*Abs(m) + 35)*H(s)**2*jacobi(\ell - 2, 7/2, Abs(m) + 2, \xi)/(4*pi) - 45*I*\gamma**2*s**2*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(2*\ell*m*H(s)**2 + 10*\ell*s**2 - 4*\ell*H(s)**2*Abs(m) - 4*m**2*H(s)**2 + 2*m*H(s)**2*Abs(m) + 5*m*H(s)**2 + 10*s**2*Abs(m) + 25*s**2 - 10*H(s)**2*Abs(m))*jacobi(\ell - 1, 5/2, Abs(m) + 1, \xi)/(4*pi) + 45*I*\gamma**2*s**Abs(m)*(25*m**2*s**6*H(s)**2 - 65*m**2*s**4*H(s)**2 + 59*m**2*s**2*H(s)**2 - 19*m**2*H(s)**2 + 75*m*s**8 - 25*m*s**6*H(s)**2*Abs(m) + 125*m*s**6*H(s)**2 - 195*m*s**6 + 65*m*s**4*H(s)**2*Abs(m) - 260*m*s**4*H(s)**2 + 177*m*s**4 - 59*m*s**2*H(s)**2*Abs(m) + 177*m*s**2*H(s)**2 - 57*m*s**2 + 19*m*H(s)**2*Abs(m) - 38*m*H(s)**2 - 125*s**8*Abs(m) - 50*s**6*H(s)**2*Abs(m) + 325*s**6*Abs(m) + 130*s**4*H(s)**2*Abs(m) - 295*s**4*Abs(m) - 118*s**2*H(s)**2*Abs(m) + 95*s**2*Abs(m) + 38*H(s)**2*Abs(m))*jacobi(\ell, 3/2, Abs(

In [36]:
# Matrix elements
dir_spec = os.path.join(dir_bg, 'Canonical')
with open(os.path.join(dir_spec, 'matrix_expr_ideal_vsHjx.json'), 'r') as fread:
    matrix_obj = json.load(fread)
M_comp = expansion.SystemMatrix.deserialize(matrix_obj["M"])
K_comp = expansion.SystemMatrix.deserialize(matrix_obj["K"])

In [37]:
K_expr[name_eqn, name_term]

InnerProduct1D(s**Abs(m + 2)*H(s)**2*jacobi(\ell', 2, Abs(m + 2) - 1/2, \xi), 45*I*\gamma**2*s**4*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(4*\ell**2 + 8*\ell*Abs(m) + 24*\ell + 4*m**2 + 24*Abs(m) + 35)*H(s)**2*jacobi(\ell - 2, 7/2, Abs(m) + 2, \xi)/(4*pi) - 45*I*\gamma**2*s**2*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(-2*\ell*m*s**2 + 2*\ell*m + 4*\ell*s**2*Abs(m) + 10*\ell*s**2 - 4*\ell*Abs(m) + 4*m**2*s**2 - 4*m**2 - 2*m*s**2*Abs(m) - 5*m*s**2 + 2*m*Abs(m) + 5*m + 20*s**2*Abs(m) + 25*s**2 - 10*Abs(m))*jacobi(\ell - 1, 5/2, Abs(m) + 1, \xi)/(4*pi) - 45*I*\gamma**2*s**Abs(m)*(s - 1)*(s + 1)*(25*m**2*s**6 - 65*m**2*s**4 + 59*m**2*s**2 - 19*m**2 - 25*m*s**6*Abs(m) + 50*m*s**6 + 65*m*s**4*Abs(m) - 140*m*s**4 - 59*m*s**2*Abs(m) + 120*m*s**2 + 19*m*Abs(m) - 38*m + 75*s**6*Abs(m) - 70*s**4*Abs(m) - 23*s**2*Abs(m) + 38*Abs(m))*jacobi(\ell, 3/2, Abs(m), \xi)/(4*pi), 1, s, 0, 1)

In [38]:
K_comp[name_eqn, name_term]

InnerProduct1D(s**Abs(m + 2)*H(s)**2*jacobi(\ell', 2, Abs(m + 2) - 1/2, \xi), 45*I*\gamma**2*s**3*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(4*\ell**2 + 8*\ell*Abs(m) + 24*\ell + 4*m**2 + 24*Abs(m) + 35)*H(s)**2*jacobi(\ell - 2, 7/2, Abs(m) + 2, \xi)/(16*pi) - 45*I*\gamma**2*s*s**Abs(m)*(s - 1)*(s + 1)*(25*s**4 - 40*s**2 + 19)*(-2*\ell*m*s**2 + 2*\ell*m + 4*\ell*s**2*Abs(m) + 10*\ell*s**2 - 4*\ell*Abs(m) + 4*m**2*s**2 - 4*m**2 - 2*m*s**2*Abs(m) - 5*m*s**2 + 2*m*Abs(m) + 5*m + 20*s**2*Abs(m) + 25*s**2 - 10*Abs(m))*jacobi(\ell - 1, 5/2, Abs(m) + 1, \xi)/(16*pi) - 45*I*\gamma**2*s**Abs(m)*(s - 1)*(s + 1)*(25*m**2*s**6 - 65*m**2*s**4 + 59*m**2*s**2 - 19*m**2 - 25*m*s**6*Abs(m) + 50*m*s**6 + 65*m*s**4*Abs(m) - 140*m*s**4 - 59*m*s**2*Abs(m) + 120*m*s**2 + 19*m*Abs(m) - 38*m + 75*s**6*Abs(m) - 70*s**4*Abs(m) - 23*s**2*Abs(m) + 38*Abs(m))*jacobi(\ell, 3/2, Abs(m), \xi)/(16*pi*s), 1, \xi, -1, 1)

In [57]:
from pg_utils.numerics import symparser

prefs = symparser.leading_powers_of(K_comp[name_eqn, name_term].opd_B, core.s, core.H)
prefs

[[Abs(m) - 1, 0], [Abs(m) + 1, 0], [Abs(m) + 3, 2]]

In [72]:
from pg_utils.numerics import matrices

prefs = matrices.InnerQuad_GaussJacobi.get_powers_pq(
    core.H, core.s, expansion.xi, K_comp[name_eqn, name_term].opd_B)
prefs

([0, 0, 2], [Abs(m) - 1, Abs(m) + 1, Abs(m) + 3], [\ell, \ell - 2, \ell - 1])

### Background field

In [9]:
from pg_utils.pg_model import bg_fields

bg_cfg = bg_fields.Background_S_l2_n1()

display_equality(core.U0_vec, bg_cfg.U0_val)
display_equality(core.B0_vec, bg_cfg.B0_val)

Eq(U_s^0(s, \phi, z), 0)

Eq(U_\phi^0(s, \phi, z), 0)

Eq(U_z^0(s, \phi, z), 0)

Eq(B_s^0(s, \phi, z), 3*sqrt(5)*\gamma*s*(-5*s**2 - 15*z**2 + 7)/(2*sqrt(pi)))

Eq(B_\phi^0(s, \phi, z), 0)

Eq(B_z^0(s, \phi, z), 3*sqrt(5)*\gamma*z*(10*s**2 + 5*z**2 - 7)/sqrt(pi))

In [10]:
bg_pg = base_utils.assemble_background(bg_cfg.B0_val, mode='PG')
# bg_pg.apply(lambda expr: expr.subs({core.H_s: core.H, core.H_s**2: core.H**2}).simplify(), inplace=True)
bg_pg.apply(lambda expr: expr.subs({core.H: core.H_s}).simplify(), inplace=True)
display_equality(core.pgvar_bg, bg_pg)

Eq(\Psi^0(s, \phi), 0)

Eq(\overline{M_{ss}}^0(s, \phi), \gamma**2*s**2*sqrt(1 - s**2)*(450*s**4 - 900*s**2 + 540)/pi)

Eq(\overline{M_{\phi\phi}}^0(s, \phi), 0)

Eq(\overline{M_{s\phi}}^0(s, \phi), 0)

Eq(\widetilde{M_{sz}}^0(s, \phi), 45*\gamma**2*s*(-25*s**6 + 30*s**4 + 3*s**2 - 8)/(4*pi))

...

Eq(B_\phi^{0+}(s, \phi), 0)

Eq(B_z^{0+}(s, \phi), 3*\gamma*sqrt(5 - 5*s**2)*(5*s**2 - 2)/sqrt(pi))

Eq(B_s^{0-}(s, \phi), 3*sqrt(5)*\gamma*s*(5*s**2 - 4)/sqrt(pi))

Eq(B_\phi^{0-}(s, \phi), 0)

Eq(B_z^{0-}(s, \phi), 3*\gamma*(2 - 5*s**2)*sqrt(5 - 5*s**2)/sqrt(pi))